In [1]:
import os
import pandas as pd

# xlsx 파일들이 들어있는 디렉토리 경로
directory_path = "Z:/02.펀드/018. 성과측정/2023/2023H1/주식매매"

# 디렉토리 내의 모든 xlsx 파일을 읽어들이기
file_list = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# 파일을 읽어들여 데이터프레임으로 변환
dataframes = []
for file in file_list:
    # 첫 번째 행을 인덱스로 설정하고자 함
    df = pd.read_excel(file, index_col=0)
    dataframes.append(df)

# 데이터프레임 병합
result = pd.concat(dataframes)

print(result)

# 펀드코드' 칼럼이 비어있는 행 삭제
result = result.dropna(subset=['펀드코드'])

# 결과를 Excel 파일로 저장
result_excel_path = os.path.join(directory_path, 'merged_stock_trade.xlsx')
result.to_excel(result_excel_path, encoding='utf-8')

print(f"병합된 결과가 {result_excel_path}에 저장되었습니다.")

           구분 거래구분        거래일     펀드코드                  펀드명          종목코드  \
No                                                                          
1    집합투자증권매수  NaN        NaT      NaN                  NaN           NaN   
2    집합투자증권매수   일반 2023-01-02  DM12001    두나미스 공모주 일반사모투자신탁  KR7251340006   
3    집합투자증권매수   일반 2023-01-02  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7229200001   
4    집합투자증권매수   일반 2023-01-02  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7252670005   
5    집합투자증권매수   일반 2023-01-02  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7229200001   
..        ...  ...        ...      ...                  ...           ...   
918      주식매도   차입 2023-06-30  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7298020009   
919      주식매도   차입 2023-06-30  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7393890009   
920      주식매도   일반 2023-06-30  DM15002         두나미스인베스트먼트홀딩  KR7340810001   
921      주식매도   일반 2023-06-30  DM15002         두나미스인베스트먼트홀딩  KR7354320004   
922      주식매도   일반 2023-06-30  DM15004      주식회사두나미스파트너스_NH  KR7354320004   

c:\Users\jtlee\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


병합된 결과가 Z:/02.펀드/018. 성과측정/2023/2023H1/주식매매\merged_stock_trade.xlsx에 저장되었습니다.


##### 당기 주식평가손익 계산 방법
1. 당기에 결산이 없는 경우(Default)
- 기말 주식평가손익 - 기초 주식평가손익
2. 중간에 결산이 있는 경우
- 기말 주식평가손익 + (결산일 주식평가손익 - 기초 주식평가손익)
3. 일부청산
- 기말 주식 보유수량이 더 적은 경우
- 기말 주식평가손익 - (기초 주식평가손익*기말 주식수/기초 주식수)
4. 추가매수
- 기말 주식 보유수량이 더 많은 경우
- 기말 주식평가손익 - 기초 주식평가손익
5. 유의할 점
- 기말에 남아있는 종목에 대해서만 평가손익 계산할 것.
- 공모주는 무조건 대표님 포지션
-> 어떻게 구분? 수기로 하는 수밖에
##### 필요한 정보
1) rawdata 파일제목에 들어가야할 정보
- 일자구분(기초, 기말, 결산), (펀드코드)
2) rawdata에서 매칭시켜야 할 정보
- 펀드 : 펀드코드 또는 펀드명으로 매칭
- 종목 : 종목코드(티커)로 매칭.(종목명은 트레이딩 팀에서 임의로 변경하는 경우에 있기에 불가)
- 매니저 : 각 종목이 어떤 매니저의 포지션인지 구분한 후 매칭

In [2]:
import os
import pandas as pd

# xlsx 파일들이 들어있는 디렉토리 경로
directory_path = "Z:/02.펀드/018. 성과측정/2023/2023H1/주식평가"

# "ye"과 "yb"를 포함하는 파일을 찾아 읽어들이기
file_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx') and ("ye" in f or "yb" in f)]

# 데이터프레임으로 변환
dfs = {}
for file_path in file_paths:
    # 파일명에서 "기말" 또는 "기초"를 추출하여 key로 사용
    key = "ye" if "ye" in file_path else "yb"
    dfs[key] = pd.read_excel(file_path, index_col=0)

# "기말"과 "기초"의 데이터프레임에서 "종목코드"가 일치하는 경우 평가손익 계산
if 'ye' in dfs and 'yb' in dfs:
    result = dfs['ye'].copy()
    result['평가손익'] = result['평가금액'] - dfs['yb']['평가금액']

    # 결과를 새로운 엑셀 파일로 저장
    result_excel_path = os.path.join(directory_path, '평가손익_차이.xlsx')
    result.to_excel(result_excel_path, encoding='utf-8')
    print(f"평가손익 차이가 {result_excel_path}에 저장되었습니다.")
else:
    print("기말 또는 기초 파일이 존재하지 않습니다.")

평가손익 차이가 Z:/02.펀드/018. 성과측정/2023/2023H1/주식평가\평가손익_차이.xlsx에 저장되었습니다.


c:\Users\jtlee\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [12]:
import os
import pandas as pd

# xlsx 파일들이 들어있는 디렉토리 경로
directory_path = "Z:/02.펀드/018. 성과측정/2023/2023H1/test/주식평가"

# 모든 xlsx 파일을 찾아 읽어들이기
file_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# 파일명에서 '펀드코드'와 '기간'을 추출하고, 각 파일의 내용을 저장할 딕셔너리 초기화
dataframes_by_fund_period = {}

# 각 파일의 데이터를 읽어들이고, '펀드코드'와 '기간'을 이용하여 딕셔너리에 데이터프레임 추가
for file_path in file_paths:
    # 파일명에서 '펀드코드'와 '기간'을 추출
    _, filename = os.path.split(file_path)
    fund_code, period = filename.replace('.xlsx', '').split('_')

    # 엑셀 파일을 데이터프레임으로 읽어들이기
    df = pd.read_excel(file_path)

    # 딕셔너리에 데이터프레임 추가
    key = (fund_code, period)
    dataframes_by_fund_period[key] = df

# 결과를 확인하거나 필요에 따라 다른 작업 수행
# dataframes_by_fund_period 딕셔너리의 키로 각 펀드코드와 기간을 사용할 수 있습니다.

# 예시: 펀드코드 'ABC'와 기간 '2022'의 데이터프레임 출력
key_to_check = ('DM13001', 'yb')
if key_to_check in dataframes_by_fund_period:
    print(dataframes_by_fund_period[key_to_check])
else:
    print(f"데이터프레임을 찾을 수 없습니다: {key_to_check}")

    No         일자     펀드코드                  펀드명          종목코드    단축코드    포지션  \
0    1 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KRY086520004   86520   LONG   
1    2 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7438580003  438580   LONG   
2    3 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7368970000  368970   LONG   
3    4 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7437780000  437780   LONG   
4    5 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7296640006  296640   LONG   
5    6 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7440200004  440200   LONG   
6    7 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7417970001  417970   LONG   
7    8 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7348340001  348340   LONG   
8    9 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7084440007   84440   LONG   
9   10 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7435870001  435870   LONG   
10  11 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7439250002  439250   LONG   
11  12 2022-12-31  DM13001  두나미스 코스닥밴처 일

In [15]:
dataframes_by_fund_period

{('DM13001',
  'yb'):     No         일자     펀드코드                  펀드명          종목코드    단축코드    포지션  \
 0    1 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KRY086520004   86520   LONG   
 1    2 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7438580003  438580   LONG   
 2    3 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7368970000  368970   LONG   
 3    4 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7437780000  437780   LONG   
 4    5 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7296640006  296640   LONG   
 5    6 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7440200004  440200   LONG   
 6    7 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7417970001  417970   LONG   
 7    8 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7348340001  348340   LONG   
 8    9 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7084440007   84440   LONG   
 9   10 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7435870001  435870   LONG   
 10  11 2022-12-31  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7439250002  439250   LONG   
 11  12

In [13]:
result_excel_path = os.path.join(directory_path, 'test.xlsx')
result.to_excel(result_excel_path, encoding='utf-8')

c:\Users\jtlee\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
